### Adhlere Coffy

### PUI Section 001

### October 13, 2015

### HW4 - Task 1: Distributions

In [ ]:
#module import
import numpy as np
import pandas as pd
import pylab as pl
from zipfile import ZipFile
from StringIO import StringIO
import urllib2
import requests
import scipy.stats 
import datetime as dt

%pylab inline

In [ ]:
#Pull in the information offline and store it in a dataframe named 'citidata'
r = requests.get('https://s3.amazonaws.com/tripdata/201505-citibike-tripdata.zip')
z = ZipFile(StringIO(r.content))
citidata = pd.read_csv(z.open("201505-citibike-tripdata.csv"))

In [ ]:
#print citidata.columns #printing the column heads for reference

### By Gender

In [ ]:
#creates two columns for male and female based on usertype and gender
citidata['ageM'] = 2015-citidata['birth year'][(citidata['usertype'] == 'Subscriber') & (citidata['gender'] == 1)]
citidata['ageF'] = 2015-citidata['birth year'][(citidata['usertype'] == 'Subscriber') & (citidata['gender'] == 2)]

In [ ]:
bins = np.arange(10,99,10) #create the variable to define the x-axis grouping #arange is faster at fulfilling the range requirements than range and/or xrange
citidata.ageM.groupby(pd.cut(citidata.ageM,bins)).agg([count_nonzero]).plot(kind='bar')
citidata.ageF.groupby(pd.cut(citidata.ageF,bins)).agg([count_nonzero]).plot(kind='bar')

In [ ]:
#create the cumulative sums for the respective gender groups
csM = citidata.ageM.groupby(pd.cut(citidata.ageM,bins)).agg([count_nonzero]).cumsum()

csF = citidata.ageF.groupby(pd.cut(citidata.ageF,bins)).agg([count_nonzero]).cumsum()

print np.abs((csM/csM.max()) - (csF/csF.max()) #gives the point to point distance between the two lines on the graph

pl.plot(bins[:-1] + 5, csM / csM.max(), label = "M")
pl.plot(bins[:-1] + 5, csF / csF.max(), label = "F")
pl.legend()

### K-S (Two Sample) Test

In [ ]:
ks=scipy.stats.ks_2samp(citidata.ageM, citidata.ageF)
print ks

#### K-S Interpretation
#### These results provide a two fold understanding of the K-S (two sample) test. Upon initial inspection the p-value being return as zero is an indication that the null hypothesis should be rejected because the two samples do not come from the same distribution. Moreover, the p-value expresses the chance (probability) that the K-S statistic is as large or larger than observed (in this case based on a score from a confidence interval). This is where the K-S (two sample) test begins to break down.

#### The K-S statistic holds a value of 0.43855. Based on the scores for various confidence levels provided in a table by Dr. Bianco (thank you!!) the K-S statistic does not satisfy the requirement set by the p-value that it is as large or larger than observed, for a 5% significance level. With this said, we fail to reject the null hypothesis that the two samples come from the same distribution. 

#### As noted in liturature, the K-S test should not be used for categorical data as the data tends to have many ties. The p-value breaks down for the K-S (two sample) test as it is calculated by evaluating the maximum point distance between the cumulative frequency distributions between the two sets.

### Pearson Test

In [ ]:
#first have to remove nan values and take a random sample of equal length for each category this was easier in an array using a loop

#the next two lines use a comprehension to loop through a list of values and only pick those that aren't NaN
ageF = np.array([i for i in citidata.ageF if not np.isnan(i)])
ageM = np.array([t for t in citidata.ageM if not np.isnan(t)])

In [ ]:
#creates a random sample of the newly created array without NaNs AND does not repeat any values
NoNaN_ageM = np.random.choice(ageM, len(ageF), replace = False) #Male array is longer than female array so it can't be the sample size

#Test to see if the array is unique
#len(NoNaN_ageM)==len(set(NoNaN_ageM)) #this does not work because set() truncates floats to 1 decimal place so it identifies resulting values as repeats

In [ ]:
scipy.stats.pearsonr(NoNaN_ageM, ageF)

#### Pearson's Interpretation
#### With such a small correlation coefficient, one can infer that the two datasets are not linearly correlated, which is not to say they are not correlated in some other fashion.

### Spearman Test

In [ ]:
scipy.stats.spearmanr(citidata.ageF, citidata.ageM, axis=0)

#### Spearman Interpretation
#### The correlation coefficience indications that the valuables are inversely correlated, e.g. as one increases the other decreases or vice versa. The p-value has a null hypothesis that the two sets have no correlation so a zero implies that there is no chance that a random sample would result in a correlation coefficient as far or further from zero as is observed here.

### Compare Day vs. Night for the Same Dataset

In [ ]:
citidata = pd.read_csv(z.open("201505-citibike-tripdata.csv"))

In [ ]:
#Have to change time format to distinguish between night & day
daytime = 6
nighttime = 19

#Loop through all data, converting the time to only hours and evaluate day/night
times = [str.split(j)[1] for j in citidata.starttime]
citidata['mystarttime']=[dt.datetime.strptime(j, "%H:%M:%S").hour for j in times]
citidata['timeD'] = 2015 - citidata['birth year'][(citidata['usertype'] == 'Subscriber') & ((citidata['mystarttime'] < nighttime) & (citidata['mystarttime'] > daytime))]
citidata['timeN'] = 2015 - citidata['birth year'][(citidata['usertype'] == 'Subscriber') & ((citidata['mystarttime'] > nighttime) & (citidata['mystarttime'] < daytime))]

In [ ]:
#calculate the cumulative frequency distribution
bins = np.arange(10,99,10)
csDay = citidata.timeD.groupby(pd.cut(citidata.timeD, bins)).agg([count_nonzero]).cumsum()
csNight = citidata.timeN.groupby(pd.cut(citidata.timeN, bins)).agg([count_nonzero]).cumsum()
print np.abs(csDay/csDay.max() - csNight/csNight.max())

pl.plot(bins[:-1] + 5, csDay/csDay.max(), label = "Day")
pl.plot(bins[:-1] + 5, csNight/csNight.max(), label = "Night")
pl.legend()

### K-S Test (Day vs. Night)

In [ ]:
scipy.stats.ks_2samp(citidata.timeD, citidata.timeN)

### Pearson Test (Day vs. Night)

In [ ]:
timeDay = np.array([i for i in citidata.timeD if not np.isnan(i)])
timeNight = np.array([t for t in citidata.timeN if not np.isnan(t)])

NoNaN_timeDay = np.random.choice(timeDay, len(timeNight), replace = False)
scipy.stats.pearsonr(NoNaN_timeDay, timeNight)

### Spearman Test (Day vs. Night)

In [ ]:
scipy.stats.spearmanr(citidata.timeD, citidata.timeN, axis = 0)